# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clusterProject').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator


In [54]:
# Loading data dropping location
dataset = spark.read.csv("hack_data.csv",header=True,inferSchema=True).drop('Location')

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [35]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [226]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')
final_data = vec_assembler.transform(dataset)
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [227]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [228]:
# Normalize each feature to have unit standard deviation.
final_data = scalerModel.transform(final_data)

In [ ]:
#Assuming 3 hackers were involve in the attack

In [229]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [230]:
#Predictions and evaluation
predictions= model.transform(final_data)
evaluator = ClusteringEvaluator()
evaluator.evaluate(predictions)

0.3068084951287429

In [231]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|   83|
|         2|   84|
|         1|  167|
+----------+-----+



In [ ]:
# The evaluation of kmeans model is low and the number of hacks for each hacker is not the same as 
#suggested by the forensic engineer. Hence it cannot be 3 hackers. Now let us check for 2 hackers

In [ ]:
#Assuming 3 hackers were involve in the attack

In [232]:
kmeans = KMeans(featuresCol='scaledFeatures',k=2)
model = kmeans.fit(final_data)
#Predictions and evaluation
predictions= model.transform(final_data)
evaluator = ClusteringEvaluator()
evaluator.evaluate(predictions)

0.6683623593283755

In [233]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [ ]:
# The evaluation of kmeans model is good and the number of hacks for each hacker is the same as 
#suggested by the forensic engineer. Hence there were only 2 hackers.